In [6]:
# Define data
α_1 = 0.1399851
γ_1 = 0.464835
R = 6378*1000 #m 
η_A = 0.29
Q = 1367 #W/m^2
EN = 40
D_r = 5.3 #m
L = 9.777 
k = 1.38064852e-23 #J/K Boltzman constant R/Na
T_s = 135 #K
B = 8
N = 2000 # pixel width
η = 0.55
λ_v = 500e-9 #nm
f = 2.2e9 #Hz
c = 3e8  #m/s
λ_c = c/f
G_R = η*π*(D_r/λ_c)^2
ρ_A = 10 #kg/m^2
ρ_p = 100 #kg/m^1.5
ρ_T = 2 #kg/m^1.5
ρ_P = 500 #kg*km
ρ_b = 0.001 #kg/J
P_l = 5 #W
μ = 3.986005e14 # m^3/s^2 standard gravitational parameter
X_r = 20 #m/px
m_c = 0.2 #kg, some required weight
η_S = 0.2 # 20% of total mass is structural mass
;

In [7]:
using JuMP

In [8]:
# Define JuMP model
m = Model()

@variables m begin
    # Subsystem specific parameters
    D_p # payload optical aperture diameter
    D_T # transmitter antenna diameter
    A # solar panel surface area

    # Mass related parameters
    m_t # total mass
    m_b # battery mass
    m_A # solar panel mass
    m_p # payload mass
    m_T # transmitter mass
    m_S # structural mass
    m_P # propulsion mass

    # Power related parameters
    P_t # total power
    P_T # transmitter power
    E_b # battery energy

    # Orbit related parameters
    h # altitude
    a # semi-major axis
    T # orbit period 
    r # worst case communications distance
    d # daylight fraction of orbit
    e # eclipse faction of orbit
    g # ground station viewing fraction of orbit
end

# TODO some of these may be better defined as NLexpressions
@NLconstraints m begin
    # Power and communications
    P_t == A*η_A*Q # all power generated by solar panel
    P_T + P_l <= d*P_t
    E_b >= P_t*e*T
    EN*L*k*T_s*R*2*π*N*B*4*r^2 == P_T*G_R*X_r*g*T*η*D_T^2

    # Payload performance
    X_r*D_p == h*λ_v

    # Orbit
    e >= 1 - d
    d <= g + 0.5
    a == R + h
    g == α_1*h^γ_1*R^(-γ_1)
    T == 2*π*a^(3/2)*μ^(-1/2)
    r^2 == h^2 + 2*R*h 

    # Mass budgets
    m_b == ρ_b*E_b
    m_p == ρ_p*D_p^1.5
    m_A == ρ_A*A
    m_T == ρ_T*D_T^1.5
    m_P == ρ_P/h
    m_S == η_S*m_t
    m_t >= m_b + m_p + m_A + m_T + m_P + m_S
end

# TODO add objective, maybe just epigraph it

# TODO to automate transform to log-space, maybe best to define 
# @GPvariables (for variables to be log-ed immediately) and 
# @GPconstraints (for constraints to transform to log-sum-exp immediately)
# can borrow some code for manipulating monomials/posynomials
# from either JuGP or from midc-signomial packages

;